In [3]:
!pip install feedparser
!pip install newspaper3k
!pip install konlpy
!pip install bs4
import feedparser
from newspaper import Article
from konlpy.tag import Okt
from bs4 import BeautifulSoup
import numpy as np
from collections import Counter
from operator import eq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 948.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=49d8b7ded05f8e13f386124c6e46247084495ffe1b1e4933ca8347052626f168
  Stored in directory: /root/.cache/pip/wheels/65/7a/a7/78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 9.9 MB/s eta 0:00:00
  Created wheel for tinyse

In [4]:
urls = ["http://rss.etnews.com/Section901.xml", 
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml", 
        "http://rss.etnews.com/Section904.xml"]

def crawl_rss(urls):
  array_rss = []
  for url in urls:
    print("[Crawl RSS] ", url)
    parse_rss = feedparser.parse(url)
    for p in parse_rss.entries:
      array_rss.append({'title':p.title, 'link':p.link})
  return array_rss

# [단계#1] 모든 RSS파일(xml형식)을 돌아다니며 기사의 제목/link 추출
list_articles = crawl_rss(urls)
print(list_articles[0]) # 첫번째 기사 출력

[Crawl RSS]  http://rss.etnews.com/Section901.xml
[Crawl RSS]  http://rss.etnews.com/Section902.xml
[Crawl RSS]  http://rss.etnews.com/Section903.xml
[Crawl RSS]  http://rss.etnews.com/Section904.xml
{'title': "코리아모빌리티, 세계 최초 허브리스 전기자전거 개발…글로벌 무대 '씽씽'", 'link': 'https://www.etnews.com/20230330000102'}


In [5]:
def crawl_article(url, language='ko'):
  print("[Crawl Article] ", url)
  a = Article(url, language = language)
  a.download()
  a.parse()
  return a.title, a.text

# [단계#2] 모든 link를 돌아다니며 본문text 추출
for article in list_articles:
  _, text = crawl_article(article['link'])
  article['text'] = text
print(list_articles[0]) # 첫번째 기사 출력

[Crawl Article]  https://www.etnews.com/20230330000102
[Crawl Article]  https://www.etnews.com/20230330000061
[Crawl Article]  https://www.etnews.com/20230330000082
[Crawl Article]  https://www.etnews.com/20230330000074
[Crawl Article]  https://www.etnews.com/20230330000083
[Crawl Article]  https://www.etnews.com/20230330000009
[Crawl Article]  https://www.etnews.com/20230330000002
[Crawl Article]  https://www.etnews.com/20230330000001
[Crawl Article]  https://www.etnews.com/20230329000275
[Crawl Article]  https://www.etnews.com/20230329000155
[Crawl Article]  https://www.etnews.com/20230329000154
[Crawl Article]  https://www.etnews.com/20230329000239
[Crawl Article]  https://www.etnews.com/20230329000288
[Crawl Article]  https://www.etnews.com/20230329000253
[Crawl Article]  https://www.etnews.com/20230329000260
[Crawl Article]  https://www.etnews.com/20230329000105
[Crawl Article]  https://www.etnews.com/20230329000274
[Crawl Article]  https://www.etnews.com/20230329000251
[Crawl Art

In [6]:
# [단계#3] 모든 본문text에서 명사 & (키워드, 빈도수) 추출

def get_keywords(text, nKeywords=10): #키워드ㅜ 추출함수, 디폴트로 10개의 키워드 추출 지정
  spliter = Okt() # konlpy에 의해서 문장을 형태소 별로 쪼갬
  nouns = spliter.nouns(text) #spliter에서 nouns함수를 불러서 텍스트를 넣으면 그 text의 명사만 추출
  count = Counter(nouns)  #추출된 명사들의 출현ㅇ빈도 추출
  list_keywords = []
  for n, c in count.most_common(nKeywords):#가장 출현빈도가 높은 명사부터 순차적으로 10번출력
    item = {'keyword': n, 'count': c} #출력할 떄는 {'keyword' , 'count'} 형식으로 출력
    list_keywords.append(item) #출력된 item들을 list_keywords에 저장
  return list_keywords #10회 반복이 끝나면 이를 list_ketwords에저장

for article in list_articles: #모든기사를 돌아다니며 text에서 명사를 추출하고 빈도에 따라 키워드 / 빈도 ㅊ출
  keywords = get_keywords(article['text']) #text를 get_keywords함수에ㅐ 넣고 결과를 그keyword와 coount로 받음(10걔)
  article['keywords'] = keywords #위의 함수 결과값인 키워드?빈도의 쌍을 KEYWORds로 만들어저장
print(list_articles[0]) # 첫번째 기사 출력

{'title': "코리아모빌리티, 세계 최초 허브리스 전기자전거 개발…글로벌 무대 '씽씽'", 'link': 'https://www.etnews.com/20230330000102', 'text': "스마트 모빌리티 스타트업 코리아모빌리티(대표 박정석)는 5월부터 3년간 총 1만5000여대의 바큇살 없는 미래형 허브리스 전기자전거 '코모 바이크(Komo Bike)'를 유럽에 수출할 계획이라고 30일 밝혔다.\n\n이 회사가 세계 최초로 개발한 코모 바이크는 바큇살이 있는 전기자전거와는 달리 휠 중앙이 뚫려 있다. 기존 전기자전거에 비해 무게중심이 낮아 주행 안정성이 높고 자체 개발한 헤어핀 모터와 인버터를 재택해 에너지 효율이 뛰어나다. 본딩 공법으로 제조해 기존 용접 대비 강도를 3배 이상 높였다. 주행거리도 늘어 1번 충전으로 약 100㎞ 거리를 이동할 수 있다. 국산 슈퍼커패시터 배터리를 적용해 30분만에 완충이 가능하다.\n\n\n\n코리아모빌리티는 유럽 수출을 위해 코모 바이크 유럽인증(CE)을 획득했다. 허브리스 타입 전기자전거 제품으로 CE인증을 받은 것도 세계에서 처음이다. 지난해 10월에는 전 세계 12개국에 지사와 법인을 운영 중인 STX와 3년 총판 계약을 맺고, 연간 5000여대 전기자전거를 유럽에 수출할 예정이다.\n\n\n\n박정석 대표는 “유럽은 자전거가 생활화돼 전기자전거가 자동차처럼 등록 관리된다”며 “유럽인들은 튼튼하면서 주행거리가 긴 친환경 전기자전거를 많이 찾는다”고 말했다.\n\n\n\n코리아모빌리티는 전기 자전거에 이어 전기오토바이 시장에도 출사표를 냈다. 최대 시속 90㎞에 주행거리 약 70㎞ 정도의 '코모 모터사이클(Komo Motorcycle)'을 출시했다. 자체 개발한 이륜차용 '헤어핀 모터'를 사용해 모터 효율을 극대했다.\n\n\n\n대부분 오토바이 차량에서 사용하는 허브모터는 모터 효율이 70% 후반에서 80% 초·중반 사이다. 코리아모빌리티가 개발한 '헤어핀 모터'는 정교한 구조 설계로 96% 이상의 효율을

In [9]:
# [단계#4] 검색어를 입력받아서 그 검색어를 가지고 있는 기사 출력
query = input()
def search_articles(query, list_keywords): # 쿼리가 키워드 리스트에 있으면 횟수출력
  nWords = 0 # 아래의 if문에 걸리지 않으면(즉, 쿼리가 키워드에 없음) 0
  for kw in list_keywords: # 키워드 리스트 검색
    noun = kw['keyword']  #쿼리와 동일한 키워드가 존재한다면
    count = kw['count'] #nWords는 그 회쉬(count)로ㅓ 갱신
    if eq(query, noun): # 쿼리와 동일한 키워드가 존재한다면
      nWords = count # nWords는 그 회수(count)로 갱신 
  return nWords #결과적으로 쿼리의 출현회수를 출력
for article in list_articles: #모든 기사를 돌아다니며 입력받은 쿼리가 몇 번 나오는지 검색(쿼리가 키워드일경우에만)
  nQuery = search_articles(query, article['keywords'])
  if nQuery != 0: # 쿼리가 키워드에 존재하지 않은 문서는 출력하지 않음
    print('[TF]:', nQuery, '[Title]:', article['title'], '[URL]:', article['link'])

존
[TF]: 12 [Title]: 지옥에서 온 애견인 '존 윅 4', 말은 더 줄이고 총은 더 쏜다 [URL]: https://www.etnews.com/20230330000009
[TF]: 12 [Title]: 지옥에서 온 애견인 '존 윅 4', 말은 더 줄이고 총은 더 쏜다 [URL]: https://www.etnews.com/20230330000009
